extract meta info of player voice from your l4d2 game folder

meta info:
- name of wav file
- length of wav
- sample rate

In [1]:
import os
from pydub import AudioSegment
import json
import tqdm

ROOT = r'D:\Download\steam\steam\steamapps\common\Left 4 Dead 2\left4dead2'
VOICE_ROOT = os.path.join(ROOT, 'sound', 'player', 'survivor', 'voice')
CHARACTERS = ['coach', 'gambler', 'mechanic', 'producer']

def read_json(path: str) -> dict:
    with open(path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return data
def write_json(path: str, obj: dict):
    with open(path, 'w', encoding='utf-8') as fp:
        json.dump(obj, fp, ensure_ascii=False, indent=4)

In [2]:
def make_meta_info(name: str):
    assert name in CHARACTERS
    meta_info = {}

    voice_folder = os.path.join(VOICE_ROOT, name)
    for wav_file in tqdm.tqdm(os.listdir(voice_folder)):
        wav_path = os.path.join(voice_folder, wav_file)
        audio: AudioSegment = AudioSegment.from_file(wav_path)
        length = len(audio)
        sr = audio.frame_rate
        meta_info[wav_file] = { 'length': length, 'sr': sr }
    write_json('../transcription/{}.meta.json'.format(name), meta_info)

for name in CHARACTERS:
    make_meta_info(name)
    print('finish handle', name)

100%|██████████| 2420/2420 [00:00<00:00, 10040.95it/s]


finish handle coach


100%|██████████| 2547/2547 [00:00<00:00, 12845.97it/s]


finish handle gambler


100%|██████████| 2616/2616 [00:00<00:00, 11847.46it/s]


finish handle mechanic


100%|██████████| 2090/2090 [00:00<00:00, 11508.67it/s]

finish handle producer


In [12]:
def align_two_folders(name: str):
    assert name in CHARACTERS
    meta_path = f'../transcription/{name}.meta.json'
    zh_path = f'../transcription/{name}_zh.json'
    meta_data = read_json(meta_path)
    zh_data = read_json(zh_path)
    zh_wavs = set(zh_data.keys())
    for wav_file in meta_data:
        if wav_file in zh_wavs:
            zh_wavs.discard(wav_file)
        else:
            print('{} not in zh_wavs'.format(wav_file))
            zh_data[wav_file] = ''
    
    print('find {} useless wav files'.format(len(zh_wavs)))
    for k in zh_wavs:
        zh_data.pop(k)
    
    write_json(zh_path, zh_data)

for name in CHARACTERS:
    align_two_folders(name)

find 0 useless wav files
find 0 useless wav files
find 0 useless wav files
find 0 useless wav files


In [9]:
obj = read_json('../model/config.json')
vits_characters = {}
for i, code in enumerate(obj['speakers']):
    vits_characters[str(code)] = i

write_json(f'../config/vit_character.json', vits_characters)